In [1]:
import pickle
import numpy as np
# import pandas as pd
import os

def save_pickle(obj, filename):
    _, ext = os.path.splitext(filename)
    if ext in ['.pkl','.p','.data']:
        with open(filename, "wb") as f:
            pickle.dump(obj, f)
    elif ext == '.npy':
        if not isinstance(obj, np.ndarray):
            obj = np.array(obj)
        np.save(filename, obj)
    else:
        pass # raise Error

def load_pickle(filename):
    _, ext = os.path.splitext(filename)
    if ext in ['.pkl','.p','.data']:
        with open(filename, "rb") as f:
            data = pickle.load(f)
        return data
    elif ext == '.npy':
        return np.load(filename)
    else:
        return None # raise Error
    
def sort_dict(m: dict) -> dict:
    return dict(sorted(m.items(), key=lambda x: x[1], reverse=True))


In [4]:
# # Generate Generation_1
# import json

# sample_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/input_sample.jsonl"
# samples = []
# with open(sample_filepath, 'r') as f:
#     for line in f:
#         samples.append(json.loads(line))

# result_filepath = "/root/pyHeter-GAT/preprocess-llm/topicGPT/script/generation_1_tmp copy.jsonl"
# results = []
# with open(result_filepath, 'r') as f:
#     for line in f:
#         results.append(json.loads(line))

# save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.jsonl"
# with open(save_filepath, 'w') as f:
#     for idx, sample in enumerate(samples):
#         result = {
#             **sample,
#             "responses": results[idx]["response"],
#         }
#         f.write(json.dumps(result, ensure_ascii=False) + '\n')


In [11]:
# import json

# sample_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm1/input_sample.jsonl"
# samples = []
# with open(sample_filepath, 'r') as f:
#     for line in f:
#         samples.append(json.loads(line))

# result_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm1/generation_1/generation_1.jsonl"
# results = []
# with open(result_filepath, 'r') as f:
#     for line in f:
#         results.append(json.loads(line))

# save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm1/generation_1/generation_1.jsonl"
# with open(save_filepath, 'w') as f:
#     for idx, sample in enumerate(samples):
#         if idx == 1000: continue
#         i = idx
#         if i > 1000: i -= 1
#         try:
#             result = {
#                 **sample,
#                 "responses": results[i]["responses"],
#             }
#         except Exception as e:
#             print(i)
#             print(e)
#         f.write(json.dumps(result, ensure_ascii=False) + '\n')

9388
list index out of range


In [4]:
# # 1. 观察Bertopic生成的话题标签数量级

# filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/cascades.data"
# cascade_dict = load_pickle(filepath)

# cnt = {}

# label_mp = {
#     0: "arts_&_culture",
#     1: "business_&_entrepreneurs",
#     2: "celebrity_&_pop_culture",
#     3: "diaries_&_daily_life",
#     4: "family",
#     5: "fashion_&_style",
#     6: "film_tv_&_video",
#     7: "fitness_&_health",
#     8: "food_&_dining",
#     9: "gaming",
#     10: "learning_&_educational",
#     11: "music",
#     12: "news_&_social_concern",
#     13: "other_hobbies",
#     14: "relationships",
#     15: "science_&_technology",
#     16: "sports",
#     17: "travel_&_adventure",
#     18: "youth_&_student_life"
# }

# for key, cascades in cascade_dict.items():
#     for b_label in cascades["label"]:
#         if b_label not in cnt: cnt[b_label] = 0
#         cnt[b_label] += 1

# # print(cnt)
# cnt[12] /= 10
# tot = sum([v for k, v in cnt.items()])
# for k, v in sort_dict(cnt).items():
#     print(label_mp[k], v, round(100*v/tot))

news_&_social_concern 43511.8 25
arts_&_culture 24074 14
diaries_&_daily_life 16859 10
science_&_technology 16049 9
business_&_entrepreneurs 12969 7
film_tv_&_video 11124 6
food_&_dining 10025 6
sports 7190 4
other_hobbies 6597 4
celebrity_&_pop_culture 6398 4
travel_&_adventure 5748 3
learning_&_educational 4090 2
music 3170 2
fitness_&_health 3134 2
family 2253 1
gaming 1557 1
relationships 1033 1
fashion_&_style 768 0
youth_&_student_life 100 0


In [7]:
# # 2. 统计最后生成的话题通道增广图的边数量级
# # 结论: 1)topic=12的图的边数量级在400w, topic=8/0的数量级在30w, 其余话题基本在几w的量级(最少的1w+)
# # 2) 从用户话题推文到话题通道的扩展量级维持在10左右, 即推文数量级*10～话题通道的数量级

# import torch.optim as optim
# from torch_geometric.data import Data
# from torch_geometric.utils import dense_to_sparse

# hedge_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_graph/topic_diffusion_graph_full_windowsize300.data"
# hedge_graph_dict = load_pickle(hedge_filepath)

# # 去掉原始的传播边
# graph_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/edges.data"
# graph = load_pickle(graph_filepath)
# n_org_edges = len(graph)

# cnt_after = {}
# for key, hgraph in hedge_graph_dict.items():
#     # print(hgraph.edge_index)
#     _, n_edges = hgraph.edge_index.shape
#     # print(n_edges)
#     cnt_after[key] = n_edges - n_org_edges
#     # print(type(hgraph))

# cnt_after[12] /= 10
# tot2 = sum([v for k, v in cnt_after.items()])
# for k,v in sort_dict(cnt_after).items():
#     # print(k, v, cnt[k], v / cnt[k])
#     print(label_mp[k], v, round(100*v/tot2))

news_&_social_concern 399217.6 31
food_&_dining 304791 24
arts_&_culture 223379 18
science_&_technology 71271 6
diaries_&_daily_life 31752 3
other_hobbies 29770 2
celebrity_&_pop_culture 27953 2
business_&_entrepreneurs 27397 2
film_tv_&_video 20163 2
fitness_&_health 18130 1
sports 15728 1
travel_&_adventure 15472 1
family 13766 1
gaming 13201 1
learning_&_educational 12768 1
music 12510 1
relationships 10685 1
fashion_&_style 10504 1
youth_&_student_life 10286 1


In [41]:
# 回想一下之前怎么构造的图

# 参考utils/graph.py中的build_heteredge_mats函数
# 1. 按照级联粒度，使用滑动窗口来构造话题通道
# P.S. 降采样了12: news_&_social_concern话题，因为它在总推文中的占比约为43w/52w
# 2. 按照话题粒度合并不同级联内的话题通道
# 3. 将每个话题的图和原始拓扑图合并
# Option: 4. 采用Motif方式做数据增强

import json
import regex
import random
from utils.log import logger
from typing import Optional
import torch
from torch_geometric.data import Data

# 1. Prepare input from TopicGPT
# P.S. start from 2000 seed users (out of 10000 total users)

result_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.jsonl"
results = []
with open(result_filepath, 'r') as f:
    for line in f:
        results.append(json.loads(line))

# 1-2. Prepare topic to interest mappings
gt_ft_mapping = load_pickle("/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/gt_ft_mapping.data")

# 2. generate simedges from interest-aware cascades

def generate_interest_cascades(data_dict: list, remove: bool = False):
    regex_pattern = r"\[(\d+)\] ([\w\s]+): ([\w\s]+)\."

    interest_cascades = {}
    for elem in data_dict:
        response = elem["responses"]
        parts = regex.compile(regex_pattern).findall(response)
        if len(parts) > 0: parts = parts[0]
        else: continue
        _, t, _ = parts[:3]
        interest = gt_ft_mapping[t]
        if interest not in interest_cascades: interest_cascades[interest] = []
        mt_info = elem["meta_info"]
        interest_cascades[interest].append((mt_info["user"], int(mt_info["ts"])))
    print(len(interest_cascades))
    
    # remove minor interests
    interest_cascades_cp = interest_cascades.copy()
    for t, cascades in interest_cascades_cp.items():
        if len(cascades) < 10:
            interest_cascades.pop(t)
    print(len(interest_cascades))

    # sort by timestamp
    for t in interest_cascades:
        interest_cascades[t] = sorted(interest_cascades[t], key=lambda x: x[1])
    return interest_cascades

def generate_simedges(interest_cascades: dict, time_distance: int = 3600 * 24 * 30):
    simedges = {}
    for interest, cascades in interest_cascades.items():
        simedges[interest] = []
        for i in range(len(cascades)-1):
            for j in range(i, len(cascades)-1):
                if cascades[j][1] - cascades[i][1] < time_distance and \
                    cascades[j][0] == cascades[i][0]:
                    simedges[interest].append((cascades[i][0], cascades[j][0]))
        # for i in range(len(cascades)-1):
        #     for j in range(max(0,i-1-window_size),min(i+1+window_size,len(cascades))): # (i-ws-1<-i->i+ws+1)
        #         if cascades[i][0] != cascades[j][0]:
        #             simedges[interest].append((cascades[i][0],cascades[j][0]))
    
    # remove abundant edges
    for interest, edges in simedges.items():
        simedges[interest] = list(set(edges))
    return simedges

# 3. convert simedges to graph
def convert_to_graph(simedges: dict, originial_edges: Optional[list] = None):
    if originial_edges:
        for interest, edges in simedges.items():
            simedges[interest] = list(set(edges) + set(originial_edges))
    
    # add self-loop edges
    for interest, edges in simedges.items():
        edges += [(u,u) for u in set([u for e in edges for u in e])]
        simedges[interest] = list(set(edges))
    
    # convert to graph
    graph_d = {}
    for interest, edges in simedges.items():
        edges = list(zip(*edges))
        edges_t = torch.LongTensor(edges)
        weight_t = torch.FloatTensor([1]*edges_t.size(1))
        graph_d[interest] = Data(edge_index=edges_t, edge_weight=weight_t)
    
    return graph_d

time_distance = 3600 * 24 * 30
save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/topic_graph_windowsize{td}.data"
save_filepath = save_filepath.format(td=time_distance / (3600 * 24))
print(save_filepath)
# graph_d = convert_to_graph(
#     generate_simedges(generate_interest_cascades(results, remove=True), window_size))
# save_pickle(graph_d, save_filepath)

interest_cascades = generate_interest_cascades(results, remove=True)

simedges = generate_simedges(interest_cascades, time_distance)
# select top20 interests
len_mp = {k:len(v) for k,v in simedges.items()}
for k in list(sort_dict(len_mp).keys())[:20]:
    print(k, len(simedges[k]))
    # simedges.pop(k)


/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/topic_graph_windowsize30.0.data
37
29
Politics 1098
Society 994
Law 726
Military 699
Health 690
Internet 422
Government 277
Religion 268
Mass_media 193
Economy 174
Entertainment 174
Technology 152
Nature 87
Business 84
Sports 69
Culture 69
Education 61
Food_and_drink 59
Human_behavior 47
Academic_disciplines 40


In [38]:
import datetime

# inspect time distance distribution
time_diff = 3600
time_distance = []
for interest, cascades in interest_cascades.items():
    for elem in cascades:
        time_distance.append(elem[1] / time_diff)
        # print(elem[1])
        # print(datetime.datetime.utcfromtimestamp(elem[1]))
        # print(datetime.datetime.utcfromtimestamp(elem[1] - time_diff))
    break

def analyse_distribution(data):
    for i in range(10):
        t = np.percentile(data, i*10)
        print(datetime.datetime.utcfromtimestamp(t * time_diff))

analyse_distribution(time_distance)
# simedges = generate_simedges(generate_interest_cascades(results, remove=True), window_size)
# select top20 interests
# len_mp = {k:len(v) for k,v in simedges.items()}
# for k in list(sort_dict(len_mp).keys())[20:]:
#     print(k, len(simedges[k]))
#     simedges.pop(k)


2019-06-21 13:01:35
2020-01-18 02:32:05.200000
2020-03-06 05:04:10.400000
2020-04-02 01:36:44.200000
2020-04-18 03:24:35.600000
2020-05-01 17:11:26
2020-05-13 23:57:51.400000
2020-05-17 13:52:18.800000
2020-05-25 09:52:55
2020-06-01 03:48:11.800000
